# New Metadata parser
Instead of looping though each individually, build the metadata dict all at once.

In [1]:
import synoptic
from synoptic.json_parsers import (
    parse_raw_variable_column,
    station_metadata_to_dataframe,
    attach_units,
)

import polars as pl

In [2]:
df = synoptic.Latest(
    # vars="air_temp,wind_speed,ozone_concentration",
    state="ut",
    # network=1,
    complete=True,
    qc=True,
    qc_checks="all",
).df()
df

🚚💨 Speedy delivery from Synoptic's latest service.
📦 Received data from 1,449 stations.
col='stid', schema=String


stid,variable,sensor_index,is_derived,value,date_time,qc_passed,qc_flags,value_string,units,id,name,elevation,latitude,longitude,mnet_id,state,timezone,elev_dem,nwszone,nwsfirezone,gacc,shortname,sgid,county,country,wims_id,cwa,period_of_record_start,period_of_record_end,providers,qc_flagged,is_restricted,is_active
str,str,u32,bool,f64,"datetime[μs, UTC]",bool,list[i64],str,str,u32,str,f64,f64,f64,u32,str,str,f64,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",list[struct[2]],bool,bool,bool
"""WBB""","""pressure""",1,false,85892.0,2024-11-20 05:29:00 UTC,true,null,null,"""Pascals""",1,"""U of U William Browning Buildi…",4806.0,40.76623,-111.84755,153,"""UT""","""America/Denver""",4727.7,"""UT105""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-20 04:50:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""U-ATAQ"",""http://air.utah.edu/""}]",false,false,true
"""GNI""","""pressure""",1,false,87058.0,2024-10-31 07:40:00 UTC,true,null,null,"""Pascals""",34,"""Gunnison Island""",4242.0,41.33216,-112.85432,153,"""UT""","""America/Denver""",4202.8,"""UT101""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Box Elder""","""US""",null,"""SLC""",1998-05-22 00:00:00 UTC,2024-11-20 04:45:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""Utah Department of Natural Resources"",""http://www.dnr.utah.gov""}, {""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",false,false,true
"""HATUT""","""pressure""",1,false,87580.0,2024-11-20 05:30:00 UTC,true,null,null,"""Pascals""",35,"""Hat Island""",4242.0,41.07073,-112.58621,153,"""UT""","""America/Denver""",4245.4,"""UT101""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Box Elder""","""US""",null,"""SLC""",1998-09-02 00:00:00 UTC,2024-11-20 04:45:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""Utah Department of Natural Resources"",""http://www.dnr.utah.gov""}, {""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",false,false,true
"""LMS""","""pressure""",1,false,87841.0,2024-11-20 05:15:00 UTC,true,null,null,"""Pascals""",36,"""Locomotive Springs""",4242.0,41.701,-112.86181,153,"""UT""","""America/Denver""",4215.9,"""UT101""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Box Elder""","""US""",null,"""SLC""",1999-07-02 00:00:00 UTC,2024-11-20 04:45:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",false,false,true
"""LMR""","""pressure""",1,false,84737.0,2024-11-20 05:15:00 UTC,true,null,null,"""Pascals""",39,"""Lakeside Mountain""",5039.0,41.06084,-112.89173,153,"""UT""","""America/Denver""",5150.9,"""UT101""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Box Elder""","""US""",null,"""SLC""",1999-12-16 00:00:00 UTC,2024-11-20 04:45:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""K41U""","""cloud_layer_3""",1,true,2590.83,2024-11-19 07:35:00 UTC,null,null,"""overcast""","""Meters""",46328,"""Manti-Ephraim Airport""",5500.0,39.33133,-111.61273,1,"""UT""","""America/Denver""",5505.2,"""UT118""","""SLC492""","""GBCC""","""ASOS/AWOS""","""GB27""","""Sanpete""","""US""",null,"""SLC""",2015-06-03 18:59:00 UTC,2024-11-20 04:35:00 UTC,"[{""National Weather Service"",""http://www.weather.gov""}]",true,false,true
"""UUSYR""","""cloud_layer_3""",1,true,2225.07,2024-11-18 15:40:00 UTC,null,null,"""N/A""","""Meters""",62231,"""Syracuse""",4217.0,41.08847,-112.1188,153,"""UT""","""America/Denver""",4215.9,"""UT104""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Davis""","""US""",null,"""SLC""",2017-03-17 19:46:00 UTC,2024-11-20 04:50:00 UTC,"[{""U of U M

In [2]:
df = synoptic.TimeSeries(
    # vars="air_temp,wind_speed,ozone_concentration",
    state="ut",
    recent="30m",
    # network=1,
    complete=True,
    qc=True,
    qc_checks="all",
).df()
df


🚚💨 Speedy delivery from Synoptic's timeseries service.
📦 Received data from 819 stations.


stid,date_time,variable,sensor_index,is_derived,value,value_sting,qc_flags,units,id,name,elevation,latitude,longitude,mnet_id,state,timezone,elev_dem,nwszone,nwsfirezone,gacc,shortname,sgid,county,country,wims_id,cwa,period_of_record_start,period_of_record_end,providers,qc_flagged,is_restricted,is_active
str,"datetime[μs, UTC]",str,u32,bool,f64,str,list[i64],str,u32,str,f64,f64,f64,u32,str,str,f64,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",list[struct[2]],bool,bool,bool
"""WBB""",2024-11-22 04:04:00 UTC,"""air_temp""",1,false,6.389,null,null,"""Celsius""",1,"""U of U William Browning Buildi…",4806.0,40.76623,-111.84755,153,"""UT""","""America/Denver""",4727.7,"""UT105""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-22 04:25:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""U-ATAQ"",""http://air.utah.edu/""}]",false,false,true
"""WBB""",2024-11-22 04:05:00 UTC,"""air_temp""",1,false,6.328,null,null,"""Celsius""",1,"""U of U William Browning Buildi…",4806.0,40.76623,-111.84755,153,"""UT""","""America/Denver""",4727.7,"""UT105""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-22 04:25:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""U-ATAQ"",""http://air.utah.edu/""}]",false,false,true
"""WBB""",2024-11-22 04:06:00 UTC,"""air_temp""",1,false,6.333,null,null,"""Celsius""",1,"""U of U William Browning Buildi…",4806.0,40.76623,-111.84755,153,"""UT""","""America/Denver""",4727.7,"""UT105""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-22 04:25:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""U-ATAQ"",""http://air.utah.edu/""}]",false,false,true
"""WBB""",2024-11-22 04:07:00 UTC,"""air_temp""",1,false,6.467,null,null,"""Celsius""",1,"""U of U William Browning Buildi…",4806.0,40.76623,-111.84755,153,"""UT""","""America/Denver""",4727.7,"""UT105""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-22 04:25:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""U-ATAQ"",""http://air.utah.edu/""}]",false,false,true
"""WBB""",2024-11-22 04:08:00 UTC,"""air_temp""",1,false,6.472,null,null,"""Celsius""",1,"""U of U William Browning Buildi…",4806.0,40.76623,-111.84755,153,"""UT""","""America/Denver""",4727.7,"""UT105""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-22 04:25:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""U-ATAQ"",""http://air.utah.edu/""}]",false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""G5565""",2024-11-22 04:08:00 UTC,"""weather_condition""",1,true,null,null,null,"""weather_condition""",252735,"""GW5565 SANDY""",4383.0,40.6,-111.9,65,"""UT""","""America/Denver""",null,"""UT105""","""SLC478""","""GBCC""","""APRSWXNET/CWOP""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",2024-11-19 19:43:00 UTC,2024-11-22 04:23:00 UTC,"[{""APRSWXNET/Citizen Weather Observer Program"",""http://www.findu.com/citizenweather""}]",false,false,true
"""G5565""",2024-11-22 04:14:00 UTC,"""weather_condition""",1,true,null,null,null,"""weather_condition""",252735,"""GW5565 SANDY""",4383.0,40.6,-111.9,65,"""UT""","""America/Denver""",null,"""UT105""","""SLC478""","""GBCC""","""APRSWXNET/CWOP""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",2024-11-19 19:43:00 UTC,2024-11-22 04:23:00 UTC,"[{""APRSWXNET/Citizen Weather Observer Program"",""http://www.findu.com/citizenweather""}]",false,false,true
"""G5565""",2024-11-22 04:18:00 UTC,"""weather_condition""",1,true,null,null,null,"""weather_condition""",252735,"""GW5565 SAN

In [3]:
df.filter(pl.col("qc_flags").is_not_null())

stid,date_time,variable,sensor_index,is_derived,value,value_sting,qc_flags,units,id,name,elevation,latitude,longitude,mnet_id,state,timezone,elev_dem,nwszone,nwsfirezone,gacc,shortname,sgid,county,country,wims_id,cwa,period_of_record_start,period_of_record_end,providers,qc_flagged,is_restricted,is_active
str,"datetime[μs, UTC]",str,u32,bool,f64,str,list[i64],str,u32,str,f64,f64,f64,u32,str,str,f64,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",list[struct[2]],bool,bool,bool
"""RTB""",2024-11-22 04:10:00 UTC,"""air_temp""",1,false,-7.072,null,[78],"""Celsius""",483,"""Rattlesnake Bench""",6468.0,38.90375,-110.56202,4,"""UT""","""America/Denver""",6476.4,"""UT121""","""SLC489""","""GBCC""","""UTAH DOT""","""GB31""","""Emery""","""US""",null,"""SLC""",1997-01-12 00:00:00 UTC,2024-11-22 04:20:00 UTC,"[{""Utah Department of Transportation"",""https://udottraffic.utah.gov/""}]",true,false,true
"""RTB""",2024-11-22 04:20:00 UTC,"""air_temp""",1,false,-6.65,null,[78],"""Celsius""",483,"""Rattlesnake Bench""",6468.0,38.90375,-110.56202,4,"""UT""","""America/Denver""",6476.4,"""UT121""","""SLC489""","""GBCC""","""UTAH DOT""","""GB31""","""Emery""","""US""",null,"""SLC""",1997-01-12 00:00:00 UTC,2024-11-22 04:20:00 UTC,"[{""Utah Department of Transportation"",""https://udottraffic.utah.gov/""}]",true,false,true
"""VRH""",2024-11-22 04:15:00 UTC,"""air_temp""",1,false,10.411,null,[105],"""Celsius""",570,"""VERNON HILL""",5761.0,40.130134,-112.384743,7,"""UT""","""America/Denver""",6148.3,"""UT102""","""SLC478""","""GBCC""","""TOOELE""","""GB25""","""Tooele""","""US""",null,"""SLC""",1998-02-10 00:00:00 UTC,2024-11-22 04:15:00 UTC,"[{""Tooele County Dept. Emergency Mngt."",""http://www.tcem.org""}]",true,false,true
"""SLCBY""",2024-11-22 04:04:00 UTC,"""air_temp""",1,false,5.6,null,[3],"""Celsius""",28396,"""NWS ""RSOIS"" Site""",4220.0,40.77223,-111.95522,8,"""UT""","""America/Denver""",4235.6,"""UT105""","""SLC478""","""GBCC""","""SNOWNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",2010-06-22 00:00:00 UTC,2024-11-22 04:23:00 UTC,"[{""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",true,false,true
"""SLCBY""",2024-11-22 04:05:00 UTC,"""air_temp""",1,false,5.6,null,[3],"""Celsius""",28396,"""NWS ""RSOIS"" Site""",4220.0,40.77223,-111.95522,8,"""UT""","""America/Denver""",4235.6,"""UT105""","""SLC478""","""GBCC""","""SNOWNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",2010-06-22 00:00:00 UTC,2024-11-22 04:23:00 UTC,"[{""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",true,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""UUPYA""",2024-11-22 04:26:00 UTC,"""wind_speed""",2,false,0.0,null,[3],"""m/s""",174940,"""UofU Playa Research Site""",4193.0,41.03028,-112.1175,153,"""UT""","""America/Denver""",4986.9,"""UT104""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Davis""","""US""",null,"""SLC""",2022-06-09 02:13:00 UTC,2024-11-22 04:15:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}]",true,false,true
"""UUPYA""",2024-11-22 04:27:00 UTC,"""wind_speed""",2,false,0.0,null,[3],"""m/s""",174940,"""UofU Playa Research Site""",4193.0,41.03028,-112.1175,153,"""UT""","""America/Denver""",4986.9,"""UT104""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Davis""","""US""",null,"""SLC""",2022-06-09 02:13:00 UTC,2024-11-22 04:15:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}]",true,false,true
"""UUPYA""",2024-11-22 04:28:00 UTC,"""wind_speed""",2,false,0.0,null,[3],"""m/s""",174940,"""UofU Playa Research Site""",4193.0,41.03028,-112.1175,153,"""UT""","""America/Denver""",4986.9,"""UT104""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Davis""","""US""",null,"""SLC""",2022-06-09 02:13:00 UTC,2024-11-22 04:15:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}]",true,false,true


In [4]:
df[1]

stid,date_time,variable,qc_flags
str,str,str,list[i64]
"""KSLC""","""2024-11-22T04:05:00Z""","""wind_direction_set_1""",[12]
"""KSLC""","""2024-11-22T04:10:00Z""","""wind_direction_set_1""",[12]
"""KSLC""","""2024-11-22T04:15:00Z""","""wind_direction_set_1""",[12]
"""KSLC""","""2024-11-22T04:20:00Z""","""wind_direction_set_1""",[12]
"""KSLC""","""2024-11-22T04:25:00Z""","""wind_direction_set_1""",[12]
…,…,…,…
"""UUPYA""","""2024-11-22T04:15:00Z""","""wind_speed_set_2""",[3]
"""UTPLB""","""2024-11-22T04:05:00Z""","""road_temp_set_4""",[3]
"""UTPLB""","""2024-11-22T04:10:00Z""","""road_temp_set_4""",[3]


In [2]:
S = synoptic.SynopticAPI(
    "timeseries",
    # vars="air_temp,wind_speed,ozone_concentration",
    # stid="wbb,ukbkb,kslc",
    recent="30m",
    state="ut",
    # network=1,
    complete=True,
    qc=True,
    qc_checks="all",
)
S

🚚💨 Speedy delivery from Synoptic's timeseries service.
📦 Received data from 904 stations.


╭─ Synoptic timeseries service ─────
│ Stations : 904
│ QC Checks: 86
╰──────────────────────────────────────╯

In [3]:
observations = []
qc = []
latency = []
sensor_variables = []

for s in S.STATION:
    observations.append({"stid": s["STID"]} | s.pop("OBSERVATIONS", {}))
    if "QC" in s:
        qc.append(
            {"stid": s["STID"], "date_time": observations[-1]["date_time"]}
            | s.pop("QC", {})
        )
    latency.append({"stid": s["STID"]} | s.pop("LATENCY", {}))
    sensor_variables.append({"stid": s["STID"]} | s.pop("SENSOR_VARIABLES", {}))

In [11]:
qc_flags = (
    pl.DataFrame(qc, infer_schema_length=None)
    .unpivot(index=["stid", "date_time"], value_name="qc_flags")
    .filter(pl.col("qc_flags").is_not_null())
    .explode("date_time", "qc_flags")
)
qc_flags.columns


['stid', 'date_time', 'variable', 'qc_flags']

In [80]:
pl.DataFrame(qc, infer_schema_length=None).select(
    "stid", "wind_direction_set_1"
).drop_nulls().explode("wind_direction_set_1")

stid,wind_direction_set_1
str,list[i64]
"""KSLC""",null
"""KSLC""",null
"""KSLC""",null
"""KSLC""",[12]
"""KSLC""",null
…,…
"""UR297""","[3, 79]"
"""UR297""","[3, 79]"
"""UR297""","[3, 79]"


In [26]:
pl.DataFrame(qc, infer_schema_length=None).filter(
    pl.any_horizontal(pl.exclude("stid").is_null())
)

stid,wind_direction_set_1,wind_speed_set_1,wind_gust_set_1,soil_temp_set_1,air_temp_set_1,relative_humidity_set_1,altimeter_set_1,solar_radiation_set_1,road_temp_set_2,surface_temp_set_1,pressure_set_1,black_carbon_concentration_set_4,soil_temp_set_2,soil_moisture_set_4,soil_temp_set_3,outgoing_radiation_sw_set_1,wind_speed_set_2,road_temp_set_4
str,list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]],list[list[i64]]
"""WBB""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""HOL""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""SNI""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""SBE""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""SB2""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""PC594""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""PC595""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""PC597""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [7]:
df = pl.DataFrame(observations, infer_schema_length=None)
df

stid,date_time,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,pressure_set_1,precip_accum_one_minute_set_1,solar_radiation_set_1,volt_set_1,volt_set_2,PM_25_concentration_set_1,flow_rate_set_1,air_flow_temperature_set_1,internal_relative_humidity_set_1,sensor_error_code_set_1,ozone_concentration_set_1,altimeter_set_1d,sea_level_pressure_set_1d,wet_bulb_temp_set_1d,wind_cardinal_direction_set_1d,wind_chill_set_1d,dew_point_temperature_set_1d,precip_accum_fifteen_minute_set_1,snow_depth_set_1,precip_accum_one_hour_set_1,altimeter_set_1,pressure_set_1d,snow_interval_set_1,precip_accum_five_minute_set_1,visibility_set_1,metar_set_1,dew_point_temperature_set_1,sea_level_pressure_set_1,cloud_layer_1_code_set_1,cloud_layer_2_code_set_1,…,sonic_air_temp_stdev_set_1,sonic_air_temp_set_1,wind_speed_set_2,wind_direction_set_2,wind_gust_set_2,air_temp_set_3,wind_cardinal_direction_set_2d,estimated_snowfall_rate_set_1,fosberg_fire_weather_index_set_1,solar_radiation_set_2,photosynthetically_active_radiation_set_2,permittivity_set_2,permittivity_set_4,permittivity_set_5,electric_conductivity_set_4,electric_conductivity_set_2,electric_conductivity_set_5,permittivity_set_3,electric_conductivity_set_3,precip_accum_30_minute_set_1,air_temp_set_5,relative_humidity_set_3,wind_speed_set_4,wind_speed_set_3,wind_speed_set_6,wind_speed_set_5,incoming_radiation_uv_set_1,outgoing_radiation_uv_set_1,particulate_concentration_set_3,particulate_concentration_set_1,particulate_concentration_set_2,dew_point_temperature_set_3d,snow_depth_set_2,evapotranspiration_set_2,sea_level_pressure_set_2d,wet_bulb_temp_set_2d,wind_chill_set_2d
str,list[str],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[str],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[str],list[f64],list[f64],list[f64],list[f64],…,list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[str],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64]
"""WBB""","[""2024-11-21T05:38:00Z"", ""2024-11-21T05:39:00Z"", … ""2024-11-21T06:05:00Z""]","[6.611, 6.556, … null]","[28.91, 28.77, … null]","[2.619, 2.583, … null]","[312.1, 335.9, … null]","[3.035, 2.84, … null]","[85822.0, 85815.0, … null]","[0.0, 0.0, … null]","[0.0, 0.0, … null]","[13.53, 13.52, … null]","[null, null, … null]","[null, null, … null]","[null, null, … null]","[null, null, … null]","[null, null, … null]","[null, null, … null]","[null, null, … 27.26]","[102410.19, 102401.84, … null]","[102325.81, 102320.95, … null]","[0.38, 0.33, … null]","[""NW"", ""NNW"", … null]","[4.72, 4.68, … null]","[-10.34, -10.45, … null]",null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""HOL""","[""2024-11-21T05:45:00Z"", ""2024-11-21T06:00:00Z""]","[4.022, 4.094]","[48.61, 45.63]","[1.096, 0.705]","[15.85, 5.96]","[3.035, 2.644]",null,null,null,"[12.9, 12.91]",null,null,null,null,null,null,null,null,null,null,"[""NNE"", ""N""]",null,"[-5.95, -6.71]","[0.0, 0.0]","[0.0, 0.0]",null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""SNI""","[""2024-11-21T06:00:00Z""]",[3.939],[35.25],null,null,null,null,null,null,[13.85],null,null,null,null,null,null,null,null,null,null,null,null,[-10

In [5]:
S.url

'https://api.synopticdata.com/v2/stations/timeseries?recent=30&state=ut&complete=1&qc=on&qc_checks=all&token=0bbe0e9fda7945a68951cc1bdebb2b0d'

In [ ]:

# TODO: Need to do something with the list of qc data
# TODO: Need to implement parsing cloud_layer

observations = []
qc = []
latency = []
sensor_variables = []

for s in S.STATION:
    observations.append({"stid": s["STID"]} | s.pop("OBSERVATIONS", {}))
    qc.append({"stid": s["STID"]} | s.pop("QC", {}))
    latency.append({"stid": s["STID"]} | s.pop("LATENCY", {}))
    sensor_variables.append({"stid": s["STID"]} | s.pop("SENSOR_VARIABLES", {}))

df = pl.DataFrame(observations, infer_schema_length=None)

In [112]:
cols_with_float = []
cols_with_string = []
cols_with_cloud_layer = []
cols_with_other = []

for col, schema in df.schema.items():
    if col in {"date_time", "stid"}:
        continue
    elif schema == pl.List(pl.Float64):
        cols_with_float.append(col)
    elif schema == pl.List(pl.String):
        cols_with_string.append(col)
    elif col.startswith("cloud_layer"):
        cols_with_cloud_layer.append(col)
    else:
        cols_with_other.append(col)
        print(f"WARNING: Unknown schema for {col=} {schema=}")

cols_with_cloud_layer

['cloud_layer_1_set_1d']

In [113]:
to_concat = []

# Unpack the float observations
if cols_with_float:
    observed_float = (
        df.select(["stid", "date_time"] + cols_with_float)
        .with_columns(
            pl.col(cols_with_float).fill_null(
                pl.lit(None, dtype=pl.Float64).repeat_by(
                    pl.col("date_time").list.len()
                )  # https://stackoverflow.com/q/78810432/2383070
            )
        )
        .explode(["date_time"] + cols_with_float)
        .unpivot(cols_with_float, index=["stid", "date_time"])
    )
    to_concat.append(observed_float)

observed_float

stid,date_time,variable,value
str,str,str,f64
"""WBB""","""2024-11-20T05:49:00Z""","""air_temp_set_1""",0.4
"""WBB""","""2024-11-20T05:50:00Z""","""air_temp_set_1""",0.433
"""WBB""","""2024-11-20T05:51:00Z""","""air_temp_set_1""",0.567
"""WBB""","""2024-11-20T05:52:00Z""","""air_temp_set_1""",0.628
"""WBB""","""2024-11-20T05:53:00Z""","""air_temp_set_1""",0.606
…,…,…,…
"""G5565""","""2024-11-20T05:53:00Z""","""wind_chill_set_2d""",null
"""G5565""","""2024-11-20T05:58:00Z""","""wind_chill_set_2d""",null
"""G5565""","""2024-11-20T06:04:00Z""","""wind_chill_set_2d""",null


In [114]:
# Unpack the string observations
#   Put values in column 'value_string'

if cols_with_string:
    observed_string = (
        df.select(["stid", "date_time"] + cols_with_string)
        .with_columns(
            pl.col(cols_with_string).fill_null(
                pl.lit(None, dtype=pl.String).repeat_by(
                    pl.col("date_time").list.len()
                )  # https://stackoverflow.com/q/78810432/2383070
            )
        )
        .explode(["date_time"] + cols_with_string)
        .unpivot(cols_with_string, index=["stid", "date_time"])
        .rename({"value": "value_sting"})
    )
    to_concat.append(observed_string)

observed_string

stid,date_time,variable,value_sting
str,str,str,str
"""WBB""","""2024-11-20T05:49:00Z""","""wind_cardinal_direction_set_1d""","""NE"""
"""WBB""","""2024-11-20T05:50:00Z""","""wind_cardinal_direction_set_1d""","""NE"""
"""WBB""","""2024-11-20T05:51:00Z""","""wind_cardinal_direction_set_1d""","""ENE"""
"""WBB""","""2024-11-20T05:52:00Z""","""wind_cardinal_direction_set_1d""","""ENE"""
"""WBB""","""2024-11-20T05:53:00Z""","""wind_cardinal_direction_set_1d""","""E"""
…,…,…,…
"""G5565""","""2024-11-20T05:53:00Z""","""wind_cardinal_direction_set_2d""",null
"""G5565""","""2024-11-20T05:58:00Z""","""wind_cardinal_direction_set_2d""",null
"""G5565""","""2024-11-20T06:04:00Z""","""wind_cardinal_direction_set_2d""",null


In [115]:
# Unpack the cloud layer.
#   Put sky_condition in 'value_sting' column
#   and height_agl in 'value' column

if cols_with_cloud_layer:
    observed_cloud_layer = (
        df.select(["stid", "date_time"] + cols_with_cloud_layer)
        .with_columns(
            pl.col(cols_with_cloud_layer).fill_null(
                pl.lit(None).repeat_by(
                    pl.col("date_time").list.len()
                )  # https://stackoverflow.com/q/78810432/2383070
            )
        )
        .explode(["date_time"] + cols_with_cloud_layer)
        .unpivot(cols_with_cloud_layer, index=["stid", "date_time"])
        .rename({"value": "value_sting"})
    )
    to_concat.append(observed_cloud_layer)

observed_cloud_layer


InvalidOperationError: `repeat_by` operation not supported for dtype `null`

In [116]:
# Join all observation values
observed = pl.concat(to_concat, how="diagonal_relaxed")

# Cast 'date_time' column from string to datetime
observed = observed.with_columns(pl.col("date_time").str.to_datetime())

# Parse the variable name
observed = observed.pipe(parse_raw_variable_column, S.UNITS)

# Join the metadata to the observed values
metadata = station_metadata_to_dataframe(S.STATION)
observed = observed.join(metadata, on="stid", how="full", coalesce=True)


In [122]:
S.url

'https://api.synopticdata.com/v2/stations/timeseries?recent=30&state=ut&complete=1&qc=on&qc_checks=all&token=0bbe0e9fda7945a68951cc1bdebb2b0d'

In [120]:
observed.filter(pl.col("qc_flagged"))

stid,date_time,variable,sensor_index,is_derived,value,value_sting,units,id,name,elevation,latitude,longitude,mnet_id,state,timezone,elev_dem,nwszone,nwsfirezone,gacc,shortname,sgid,county,country,wims_id,cwa,period_of_record_start,period_of_record_end,providers,qc_flagged,is_restricted,is_active
str,"datetime[μs, UTC]",str,u32,bool,f64,str,str,u32,str,f64,f64,f64,u32,str,str,f64,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",list[struct[2]],bool,bool,bool
"""KPVU""",2024-11-20 05:50:00 UTC,"""air_temp""",1,false,-3.0,null,"""Celsius""",58,"""Provo Municipal Airport""",4495.0,40.2239,-111.7253,1,"""UT""","""America/Denver""",4491.5,"""UT106""","""SLC478""","""GBCC""","""ASOS/AWOS""","""GB25""","""Utah""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-20 05:45:00 UTC,"[{""National Weather Service"",""http://www.weather.gov""}]",true,false,true
"""KPVU""",2024-11-20 05:55:00 UTC,"""air_temp""",1,false,-3.0,null,"""Celsius""",58,"""Provo Municipal Airport""",4495.0,40.2239,-111.7253,1,"""UT""","""America/Denver""",4491.5,"""UT106""","""SLC478""","""GBCC""","""ASOS/AWOS""","""GB25""","""Utah""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-20 05:45:00 UTC,"[{""National Weather Service"",""http://www.weather.gov""}]",true,false,true
"""KPVU""",2024-11-20 05:56:00 UTC,"""air_temp""",1,false,-3.9,null,"""Celsius""",58,"""Provo Municipal Airport""",4495.0,40.2239,-111.7253,1,"""UT""","""America/Denver""",4491.5,"""UT106""","""SLC478""","""GBCC""","""ASOS/AWOS""","""GB25""","""Utah""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-20 05:45:00 UTC,"[{""National Weather Service"",""http://www.weather.gov""}]",true,false,true
"""KPVU""",2024-11-20 06:00:00 UTC,"""air_temp""",1,false,-4.0,null,"""Celsius""",58,"""Provo Municipal Airport""",4495.0,40.2239,-111.7253,1,"""UT""","""America/Denver""",4491.5,"""UT106""","""SLC478""","""GBCC""","""ASOS/AWOS""","""GB25""","""Utah""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-20 05:45:00 UTC,"[{""National Weather Service"",""http://www.weather.gov""}]",true,false,true
"""KPVU""",2024-11-20 06:05:00 UTC,"""air_temp""",1,false,-5.0,null,"""Celsius""",58,"""Provo Municipal Airport""",4495.0,40.2239,-111.7253,1,"""UT""","""America/Denver""",4491.5,"""UT106""","""SLC478""","""GBCC""","""ASOS/AWOS""","""GB25""","""Utah""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-20 05:45:00 UTC,"[{""National Weather Service"",""http://www.weather.gov""}]",true,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""AV997""",2024-11-20 06:07:00 UTC,"""wind_cardinal_direction""",2,true,null,null,"""wind_cardinal_direction""",232230,"""NW5W-12 Suncrest""",6201.0,40.47583,-111.84533,65,"""UT""","""America/Denver""",null,"""UT111""","""SLC478""","""GBCC""","""APRSWXNET/CWOP""","""GB25""","""Utah""","""US""",null,"""SLC""",2024-03-03 20:40:00 UTC,2024-11-20 05:47:00 UTC,"[{""APRSWXNET/Citizen Weather Observer Program"",""http://www.findu.com/citizenweather""}]",true,false,true
"""AV997""",2024-11-20 06:12:00 UTC,"""wind_cardinal_direction""",2,true,null,null,"""wind_cardinal_direction""",232230,"""NW5W-12 Suncrest""",6201.0,40.47583,-111.84533,65,"""UT""","""America/Denver""",null,"""UT111""","""SLC478""","""GBCC""","""APRSWXNET/CWOP""","""GB25""","""Utah""","""US""",null,"""SLC""",2024-03-03 20:40:00 UTC,2024-11-20 05:47:00 UTC,"[{""APRSWXNET/Citizen Weather Observer Program"",""http://www.findu.com/citizenweather""}]",true,false,true
"""G5229""",2024-11-20 05:50:00 UTC,"""wind_cardinal_direction""",2,true,null,null,"""wind_cardinal_direction""",242416,"""GW5229 LINDON""",4629.0,40.33317,-111.72767,65,"""UT""","""America/Denver""",null,"""UT106""","""SLC478""","""GBCC""","""APRSWXNET/CWOP""","""GB25""","""Utah""","""US""",null,"""SLC""",2024-08-06 17:36:00 UTC,2024-11-20 05:50:00 UTC,"[{""APRSWXNET/Citizen Weather Observer Program"",""http://www.findu.com/citizenweather""}]",true,false,true


In [ ]:
(
    df.select(["stid", "date_time"] + cols_with_cloud_layer).with_columns(
        pl.col(cols_with_cloud_layer).fill_null(
            pl.struct({'sky_condition':None, "height_agl":None})), dtype=pl.Boolean).repeat_by(
                pl.col("date_time").list.len()
            )  # https://stackoverflow.com/q/78810432/2383070
        )
    )
)

SyntaxError: unmatched ')' (1834039484.py, line 9)

In [3]:
import polars as pl


def unnest_period_of_record(
    df: pl.DataFrame | pl.LazyFrame,
) -> pl.DataFrame | pl.LazyFrame:
    """Un-nest the PERIOD_OF_RECORD column struct."""
    return df.with_columns(
        pl.struct(
            pl.col("PERIOD_OF_RECORD")
            .struct.field("start")
            .cast(pl.String)
            .str.to_datetime(time_zone="UTC")
            .alias("PERIOD_OF_RECORD_START"),
            pl.col("PERIOD_OF_RECORD")
            .struct.field("end")
            .cast(pl.String)
            .str.to_datetime(time_zone="UTC")
            .alias("PERIOD_OF_RECORD_END"),
        ).alias("PERIOD_OF_RECORD"),
    ).unnest("PERIOD_OF_RECORD")


def station_metadata_to_dataframe(STATION: list[dict]):
    """From STATION, produce the metadata DataFrame."""
    a = []
    for metadata in STATION:
        metadata = metadata.copy()
        metadata.pop("OBSERVATIONS", None)
        metadata.pop("SENSOR_VARIABLES", None)
        metadata.pop("LATENCY", None)
        metadata.pop("QC", None)
        a.append(metadata)
    df = pl.DataFrame(a, infer_schema_length=None).lazy()
    df = df.with_columns(
        pl.col("STID").cast(pl.String),
        pl.col("ID", "MNET_ID").cast(pl.UInt32),
        pl.col("ELEVATION", "LATITUDE", "LONGITUDE").cast(pl.Float64),
        is_active=pl.when(pl.col("STATUS") == "ACTIVE")
        .then(True)
        .otherwise(pl.when(pl.col("STATUS") == "INACTIVE").then(False)),
    ).drop("UNITS", "STATUS")

    if "RESTRICTED" in df.collect_schema().names():
        df = df.rename({"RESTRICTED": "is_restricted"})

    if "ELEV_DEM" in df.collect_schema().names():
        # This isn't in the Latency request
        df = df.with_columns(pl.col("ELEV_DEM").cast(pl.Float64))

    df = df.pipe(unnest_period_of_record)
    df = df.rename({i: i.lower() for i in df.collect_schema().names()})

    return df.collect()


def NEW_parse_stations_latest_nearesttime(STATION):
    pass


In [4]:
# Unpack Latest/Nearest time JSON into parts

observations = []
qc = []
latency = []
sensor_variables = []

for s in S.STATION:
    observations.append({"stid": s["STID"]} | s.pop("OBSERVATIONS", {}))
    qc.append({"stid": s["STID"]} | s.pop("qc", {}))
    latency.append({"stid": s["STID"]} | s.pop("latency", {}))
    sensor_variables.append({"stid": s["STID"]} | s.pop("sensor_variables", {}))


# Get Metadata DataFrame
metadata = station_metadata_to_dataframe(S.STATION)

# Get Observations DataFrame (needs more processing)
df = pl.DataFrame(observations, infer_schema_length=None)

In [5]:
# BUG: Synoptic API ozone_concentration_value_1, the value is returned as string and not float
df = df.with_columns(
    pl.struct(
        [
            pl.col("ozone_concentration_value_1")
            .struct.field("value")
            .replace("", None)
            .cast(pl.Float64),
            pl.col("ozone_concentration_value_1").struct.field("date_time"),
        ]
    ).alias("ozone_concentration_value_1")
)


In [6]:
# Separate columns by value type
# TODO: Still need to handle sky_condition types

cols_with_float = []
cols_with_string = []
cols_with_cloud_layer = []
cols_with_other = []
for col, schema in df.schema.items():
    if hasattr(schema, "fields"):
        if pl.Field("value", pl.Float64) in schema.fields:
            cols_with_float.append(col)
        elif pl.Field("value", pl.String) in schema.fields:
            cols_with_string.append(col)
        elif col.startswith("cloud_layer"):
            cols_with_cloud_layer.append(col)
        elif pl.Field("value", pl.Struct) in schema.fields:
            cols_with_other.append(col)
            print(f"WARNING: Unknown struct for {col=} {schema=}")
    else:
        print(f"{col=}, {schema=}")

col='stid', schema=String


In [9]:
# Unpack the Float observations
observed_float = (
    df.select(["stid"] + cols_with_float)
    .select("stid", "^.*value.*$")
    .unpivot(index="stid")
    # .with_columns(
    #    pl.col("variable").str.extract_groups(
    #        r"(?<variable>.+)_value_(?<sensor_index>\d)(?<is_derived>d?)"
    #    )
    # )
    # .unnest("variable")
    # .with_columns(
    #    pl.col("is_derived") == "d",
    #    pl.col("sensor_index").cast(pl.UInt32),
    #    pl.col("variable").replace(S.UNITS).alias("units"),
    # )
    .unnest("value")
    .with_columns(pl.col("date_time").str.to_datetime())
    .drop_nulls()
)
observed_float

stid,variable,value,date_time,qc
str,str,f64,"datetime[μs, UTC]",struct[2]
"""WBB""","""pressure_value_1""",85878.0,2024-11-20 04:55:00 UTC,"{""passed"",null}"
"""GNI""","""pressure_value_1""",87058.0,2024-10-31 07:40:00 UTC,"{""passed"",null}"
"""HATUT""","""pressure_value_1""",87580.0,2024-11-20 04:45:00 UTC,"{""passed"",null}"
"""LMS""","""pressure_value_1""",87821.0,2024-11-20 04:45:00 UTC,"{""passed"",null}"
"""LMR""","""pressure_value_1""",84728.0,2024-11-20 04:45:00 UTC,"{""passed"",null}"
…,…,…,…,…
"""UGSPG""","""evapotranspiration_value_2""",0.0,2024-11-20 04:30:00 UTC,"{""passed"",null}"
"""UUCMF""","""evapotranspiration_value_2""",0.0051,2024-11-20 04:30:00 UTC,"{""passed"",null}"
"""UUPYF""","""evapotranspiration_value_2""",0.0,2024-11-20 04:30:00 UTC,"{""passed"",null}"


In [11]:
# Unpack the string observations
observed_string = (
    df.select(["stid"] + cols_with_string)
    .select("stid", "^.*value.*$")
    .unpivot(index="stid")
    # .with_columns(
    #    pl.col("variable").str.extract_groups(
    #        r"(?<variable>.+)_value_(?<sensor_index>\d)(?<is_derived>d?)"
    #    )
    # )
    # .unnest("variable")
    # .with_columns(
    #    pl.col("is_derived") == "d",
    #    pl.col("sensor_index").cast(pl.UInt32),
    #    pl.col("variable").replace(S.UNITS).alias("units"),
    # )
    .unnest("value")
    .rename({"value": "value_string"})
    .with_columns(pl.col("date_time").str.to_datetime())
    .drop_nulls()
)
observed_string


stid,variable,value_string,date_time,qc
str,str,str,"datetime[μs, UTC]",struct[1]
"""KSLC""","""metar_value_1""","""METAR KSLC 200454Z 18006KT 10S…",2024-11-20 04:54:00 UTC,"{""passed""}"
"""KU42""","""metar_value_1""","""METAR KU42 200435Z AUTO 17005K…",2024-11-20 04:35:00 UTC,"{""passed""}"
"""KHIF""","""metar_value_1""","""METAR KHIF 200455Z AUTO 12011K…",2024-11-20 04:55:00 UTC,"{""passed""}"
"""KOGD""","""metar_value_1""","""METAR KOGD 200453Z AUTO 16007K…",2024-11-20 04:53:00 UTC,"{""passed""}"
"""KBMC""","""metar_value_1""","""METAR KBMC 200435Z AUTO 13003K…",2024-11-20 04:35:00 UTC,"{""passed""}"
…,…,…,…,…
"""KFOM""","""metar_value_1""","""METAR KFOM 200435Z AUTO 10SM C…",2024-11-20 04:35:00 UTC,"{""passed""}"
"""K41U""","""metar_value_1""","""METAR K41U 200435Z AUTO 03004K…",2024-11-20 04:35:00 UTC,"{""passed""}"
"""KSPK""","""metar_value_1""","""METAR KSPK 200435Z AUTO 13009K…",2024-11-20 04:35:00 UTC,"{""passed""}"


In [12]:
# Unpack the cloud layer
observed_cloud_layer = (
    (
        df.select(["stid"] + cols_with_cloud_layer)
        .select("stid", "^.*value.*$")
        .unpivot(index="stid")
        # .with_columns(
        #    pl.col("variable").str.extract_groups(
        #        r"(?<variable>.+)_value_(?<sensor_index>\d)(?<is_derived>d?)"
        #    )
        # )
        # .unnest("variable")
        # .with_columns(
        #    pl.col("is_derived") == "d",
        #    pl.col("sensor_index").cast(pl.UInt32),
        #    pl.col("variable").replace(S.UNITS).alias("units"),
        # )
        .unnest("value")
        .rename({"value": "value_cloud_layer"})
        .with_columns(pl.col("date_time").str.to_datetime())
        .drop_nulls()
    )
    .unnest("value_cloud_layer")
    .rename({"sky_condition": "value_string", "height_agl": "value"})
)
observed_cloud_layer


stid,variable,date_time,value_string,value
str,str,"datetime[μs, UTC]",str,f64
"""KSLC""","""cloud_layer_1_value_1d""",2024-11-20 04:54:00 UTC,"""clear""",null
"""KU42""","""cloud_layer_1_value_1d""",2024-11-20 04:35:00 UTC,"""clear""",null
"""KHIF""","""cloud_layer_1_value_1d""",2024-11-20 04:55:00 UTC,"""clear""",null
"""KOGD""","""cloud_layer_1_value_1d""",2024-11-20 04:53:00 UTC,"""clear""",null
"""KBMC""","""cloud_layer_1_value_1d""",2024-11-20 04:35:00 UTC,"""clear""",null
…,…,…,…,…
"""K41U""","""cloud_layer_3_value_1d""",2024-11-19 07:35:00 UTC,"""overcast""",2590.83
"""UUSYR""","""cloud_layer_3_value_1d""",2024-11-18 15:40:00 UTC,"""N/A""",2225.07
"""KSPK""","""cloud_layer_3_value_1d""",2024-11-19 15:35:00 UTC,"""broken""",2286.03


In [17]:
# Join all observation values

observed = pl.concat(
    [observed_float, observed_string, observed_cloud_layer], how="diagonal_relaxed"
)

In [18]:
# Join the metadata to the observed values

observed = observed.join(metadata, on="stid", how="full", coalesce=True)

# Pase the variable name
observed = (
    observed.with_columns(
        pl.col("variable").str.extract_groups(
            r"(?<variable>.+)_value_(?<sensor_index>\d)(?<is_derived>d?)"
        )
    )
    .unnest("variable")
    .with_columns(
        pl.col("is_derived") == "d",
        pl.col("sensor_index").cast(pl.UInt32),
        pl.col("variable").replace(S.UNITS).alias("units"),
    )
)

if "qc" in observed.columns:
    observed = (
        observed.unnest("qc")
        .rename({"status": "qc_passed"})
        .with_columns(
            pl.col("qc_passed").replace_strict({"failed": False, "passed": True})
        )
    )

observed

stid,variable,sensor_index,is_derived,value,date_time,qc_passed,qc_flags,value_string,id,name,elevation,latitude,longitude,mnet_id,state,timezone,elev_dem,nwszone,nwsfirezone,gacc,shortname,sgid,county,country,wims_id,cwa,period_of_record_start,period_of_record_end,providers,qc_flagged,is_restricted,is_active,units
str,str,u32,bool,f64,"datetime[μs, UTC]",bool,list[i64],str,u32,str,f64,f64,f64,u32,str,str,f64,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",list[struct[2]],bool,bool,bool,str
"""WBB""","""pressure""",1,false,85878.0,2024-11-20 04:55:00 UTC,true,null,null,1,"""U of U William Browning Buildi…",4806.0,40.76623,-111.84755,153,"""UT""","""America/Denver""",4727.7,"""UT105""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Salt Lake""","""US""",null,"""SLC""",1997-01-01 00:00:00 UTC,2024-11-20 04:50:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""U-ATAQ"",""http://air.utah.edu/""}]",false,false,true,"""Pascals"""
"""GNI""","""pressure""",1,false,87058.0,2024-10-31 07:40:00 UTC,true,null,null,34,"""Gunnison Island""",4242.0,41.33216,-112.85432,153,"""UT""","""America/Denver""",4202.8,"""UT101""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Box Elder""","""US""",null,"""SLC""",1998-05-22 00:00:00 UTC,2024-11-20 04:45:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""Utah Department of Natural Resources"",""http://www.dnr.utah.gov""}, {""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",false,false,true,"""Pascals"""
"""HATUT""","""pressure""",1,false,87580.0,2024-11-20 04:45:00 UTC,true,null,null,35,"""Hat Island""",4242.0,41.07073,-112.58621,153,"""UT""","""America/Denver""",4245.4,"""UT101""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Box Elder""","""US""",null,"""SLC""",1998-09-02 00:00:00 UTC,2024-11-20 04:45:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""Utah Department of Natural Resources"",""http://www.dnr.utah.gov""}, {""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",false,false,true,"""Pascals"""
"""LMS""","""pressure""",1,false,87821.0,2024-11-20 04:45:00 UTC,true,null,null,36,"""Locomotive Springs""",4242.0,41.701,-112.86181,153,"""UT""","""America/Denver""",4215.9,"""UT101""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Box Elder""","""US""",null,"""SLC""",1999-07-02 00:00:00 UTC,2024-11-20 04:45:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",false,false,true,"""Pascals"""
"""LMR""","""pressure""",1,false,84728.0,2024-11-20 04:45:00 UTC,true,null,null,39,"""Lakeside Mountain""",5039.0,41.06084,-112.89173,153,"""UT""","""America/Denver""",5150.9,"""UT101""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Box Elder""","""US""",null,"""SLC""",1999-12-16 00:00:00 UTC,2024-11-20 04:45:00 UTC,"[{""U of U MesoWest Group"",""http://meso1.chpc.utah.edu/mesowest_overview/""}, {""SLC WFO/NWS Western Region"",""http://www.wrh.noaa.gov/slc""}]",false,false,true,"""Pascals"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""K41U""","""cloud_layer_3""",1,true,2590.83,2024-11-19 07:35:00 UTC,null,null,"""overcast""",46328,"""Manti-Ephraim Airport""",5500.0,39.33133,-111.61273,1,"""UT""","""America/Denver""",5505.2,"""UT118""","""SLC492""","""GBCC""","""ASOS/AWOS""","""GB27""","""Sanpete""","""US""",null,"""SLC""",2015-06-03 18:59:00 UTC,2024-11-20 04:35:00 UTC,"[{""National Weather Service"",""http://www.weather.gov""}]",false,false,true,"""Meters"""
"""UUSYR""","""cloud_layer_3""",1,true,2225.07,2024-11-18 15:40:00 UTC,null,null,"""N/A""",62231,"""Syracuse""",4217.0,41.08847,-112.1188,153,"""UT""","""America/Denver""",4215.9,"""UT104""","""SLC478""","""GBCC""","""UUNET""","""GB25""","""Davis""","""US""",null,"""SLC""",2017-03-17 19:46:00 UTC,2024-11-20 04:50:00 UTC,"[{""U of U MesoWest Grou

In [16]:
observed["units"]

units
str
"""Pascals"""
"""Pascals"""
"""Pascals"""
"""Pascals"""
"""Pascals"""
…
"""Meters"""
"""Meters"""
"""Meters"""


## New Latest service parser

In [50]:
from synoptic import SynopticAPI
from synoptic.json_parsers import station_metadata_to_dataframe
import polars as pl
from datetime import timedelta, datetime

In [63]:
S = SynopticAPI(
    "latency",
    radius="UKBKB,10",
    vars="air_temp",
    start=datetime(2024, 1, 1),
    end=datetime(2024, 1, 2),
    stats="all",
)
S.json

🚚💨 Speedy delivery from Synoptic's latency service.
📦 Received data from 19 stations.


{'STATION': [{'ID': '10331',
   'STID': 'QSF',
   'NAME': 'Spanish Fork',
   'ELEVATION': '4537.0',
   'LATITUDE': '40.13630',
   'LONGITUDE': '-111.6602',
   'STATUS': 'ACTIVE',
   'MNET_ID': '9',
   'STATE': 'UT',
   'TIMEZONE': 'America/Denver',
   'RESTRICTED_DATA': '0',
   'DISTANCE': 3.12,
   'PERIOD_OF_RECORD': {'start': '2004-06-09T00:00:00Z',
    'end': '2024-12-01T04:00:00Z'},
   'UNITS': {'position': 'm', 'elevation': 'ft'},
   'LATENCY': {'date_time': ['2024-01-01T00:00:00Z',
     '2024-01-01T01:00:00Z',
     '2024-01-01T02:00:00Z',
     '2024-01-01T03:00:00Z',
     '2024-01-01T04:00:00Z',
     '2024-01-01T05:00:00Z',
     '2024-01-01T06:00:00Z',
     '2024-01-01T07:00:00Z',
     '2024-01-01T08:00:00Z',
     '2024-01-01T09:00:00Z',
     '2024-01-01T10:00:00Z',
     '2024-01-01T11:00:00Z',
     '2024-01-01T12:00:00Z',
     '2024-01-01T13:00:00Z',
     '2024-01-01T14:00:00Z',
     '2024-01-01T15:00:00Z',
     '2024-01-01T16:00:00Z',
     '2024-01-01T17:00:00Z',
     '2024-01-

In [67]:
dfs = []
for station in S.STATION:
    latency = (
        pl.DataFrame(station["LATENCY"])
        .with_columns(
            pl.col("date_time").str.to_datetime(),
            pl.duration(minutes="values").alias("latency"),
        )
        .drop("values")
    )

    dfs.append(latency)

df = pl.concat(dfs, how="diagonal_relaxed")
df

date_time,latency
"datetime[μs, UTC]",duration[μs]
2024-01-01 00:00:00 UTC,25m
2024-01-01 01:00:00 UTC,25m
2024-01-01 02:00:00 UTC,25m
2024-01-01 03:00:00 UTC,25m
2024-01-01 04:00:00 UTC,25m
…,…
2024-01-01 23:20:00 UTC,6m
2024-01-01 23:30:00 UTC,6m
2024-01-01 23:40:00 UTC,6m


In [68]:
observations = []
qc = []
latency = []
sensor_variables = []

for s in S.STATION:
    observations.append({"stid": s["STID"]} | s.pop("OBSERVATIONS", {}))
    qc.append({"stid": s["STID"]} | s.pop("QC", {}))
    latency.append({"stid": s["STID"]} | s.pop("LATENCY", {}))
    sensor_variables.append({"stid": s["STID"]} | s.pop("SENSOR_VARIABLES", {}))


In [71]:
df = pl.DataFrame(latency).explode("date_time", "values")

# Join the metadata to the observed values
metadata = station_metadata_to_dataframe(S.STATION)
df = df.join(metadata, on="stid", how="full", coalesce=True)

df

stid,date_time,values,id,name,elevation,latitude,longitude,mnet_id,state,timezone,restricted_data,distance,period_of_record_start,period_of_record_end,statistics,is_active
str,str,i64,u32,str,f64,f64,f64,u32,str,str,str,f64,"datetime[μs, UTC]","datetime[μs, UTC]",struct[10],bool
"""QSF""","""2024-01-01T00:00:00Z""",25,10331,"""Spanish Fork""",4537.0,40.1363,-111.6602,9,"""UT""","""America/Denver""","""0""",3.12,2004-06-09 00:00:00 UTC,2024-12-01 04:00:00 UTC,"{""202401010000"",""202401020000"",""2024-01-01T00:00:00Z"",""2024-01-01T00:00:00Z"",25.0,25.0,25.0,25.0,0.0,25.0}",true
"""QSF""","""2024-01-01T01:00:00Z""",25,10331,"""Spanish Fork""",4537.0,40.1363,-111.6602,9,"""UT""","""America/Denver""","""0""",3.12,2004-06-09 00:00:00 UTC,2024-12-01 04:00:00 UTC,"{""202401010000"",""202401020000"",""2024-01-01T00:00:00Z"",""2024-01-01T00:00:00Z"",25.0,25.0,25.0,25.0,0.0,25.0}",true
"""QSF""","""2024-01-01T02:00:00Z""",25,10331,"""Spanish Fork""",4537.0,40.1363,-111.6602,9,"""UT""","""America/Denver""","""0""",3.12,2004-06-09 00:00:00 UTC,2024-12-01 04:00:00 UTC,"{""202401010000"",""202401020000"",""2024-01-01T00:00:00Z"",""2024-01-01T00:00:00Z"",25.0,25.0,25.0,25.0,0.0,25.0}",true
"""QSF""","""2024-01-01T03:00:00Z""",25,10331,"""Spanish Fork""",4537.0,40.1363,-111.6602,9,"""UT""","""America/Denver""","""0""",3.12,2004-06-09 00:00:00 UTC,2024-12-01 04:00:00 UTC,"{""202401010000"",""202401020000"",""2024-01-01T00:00:00Z"",""2024-01-01T00:00:00Z"",25.0,25.0,25.0,25.0,0.0,25.0}",true
"""QSF""","""2024-01-01T04:00:00Z""",25,10331,"""Spanish Fork""",4537.0,40.1363,-111.6602,9,"""UT""","""America/Denver""","""0""",3.12,2004-06-09 00:00:00 UTC,2024-12-01 04:00:00 UTC,"{""202401010000"",""202401020000"",""2024-01-01T00:00:00Z"",""2024-01-01T00:00:00Z"",25.0,25.0,25.0,25.0,0.0,25.0}",true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""FG030""","""2024-01-01T23:20:00Z""",6,221466,"""Payson East""",4751.0,40.03216,-111.69474,138,"""UT""","""America/Denver""","""0""",5.81,2023-10-18 12:10:00 UTC,2024-12-01 04:20:00 UTC,"{""202401010000"",""202401020000"",""2024-01-01T03:10:00Z"",""2024-01-01T00:00:00Z"",145.0,6.206897,26.0,6.0,1.845391,6.0}",true
"""FG030""","""2024-01-01T23:30:00Z""",6,221466,"""Payson East""",4751.0,40.03216,-111.69474,138,"""UT""","""America/Denver""","""0""",5.81,2023-10-18 12:10:00 UTC,2024-12-01 04:20:00 UTC,"{""202401010000"",""202401020000"",""2024-01-01T03:10:00Z"",""2024-01-01T00:00:00Z"",145.0,6.206897,26.0,6.0,1.845391,6.0}",true
"""FG030""","""2024-01-01T23:40:00Z""",6,221466,"""Payson East""",4751.0,40.03216,-111.69474,138,"""UT""","""America/Denver""","""0""",5.81,2023-10-18 12:10:00 UTC,2024-12-01 04:20:00 UTC,"{""202401010000"",""202401020000"",""2024-01-01T03:10:00Z"",""2024-01-01T00:00:00Z"",145.0,6.206897,26.0,6.0,1.845391,6.0}",true


In [ ]:
observations[0]

{'stid': 'BLUN2',
 'total_precip_value_1': 20.574,
 'ob_start_time_1': '2024-10-09T23:37:00Z',
 'ob_end_time_1': '2024-10-31T23:37:00Z',
 'count_1': 520}

In [1]:
from synoptic.services import Networks, Latest, NearestTime
import polars as pl
from datetime import datetime, timedelta

s = NearestTime(
    radius="KMRY,10",
    attime=datetime(2024, 1, 1),
    within=timedelta(minutes=60),
    vars="wind_speed",
    qc=True,
    qc_checks="all",
)
s

🚚💨 Speedy delivery from Synoptic's nearesttime service.
📦 Received data from 28 stations.


╭─ Synoptic nearesttime service ─────
│ Stations : 28
│ QC Checks: 86
╰──────────────────────────────────────╯

In [2]:
len(s.STATION), s.df()["stid"].n_unique(), s.SUMMARY["NUMBER_OF_OBJECTS"]

col='stid', schema=String


(28, 28, 28)

In [3]:
s.df()

stid,variable,sensor_index,is_derived,value,date_time,qc_passed,qc_flags,units,id,name,elevation,latitude,longitude,mnet_id,state,timezone,elev_dem,distance,period_of_record_start,period_of_record_end,qc_flagged,is_restricted,is_active
str,str,u32,bool,f64,"datetime[μs, UTC]",bool,list[i64],str,u32,str,f64,f64,f64,u32,str,str,f64,f64,"datetime[μs, UTC]","datetime[μs, UTC]",bool,bool,bool
"""KMRY""","""wind_speed""",1,false,0.0,2024-01-01 00:00:00 UTC,true,null,"""m/s""",276,"""Monterey Regional Airport""",167.0,36.59047,-121.84875,1,"""CA""","""America/Los_Angeles""",170.6,0.0,1997-04-12 00:00:00 UTC,2024-12-03 04:00:00 UTC,false,false,true
"""KOAR""","""wind_speed""",1,false,2.058,2023-12-31 23:55:00 UTC,true,null,"""m/s""",306,"""FORT ORD/FRITZSCHE""",135.0,36.68,-121.77,1,"""CA""","""America/Los_Angeles""",144.4,7.57,2020-11-04 19:04:00 UTC,2024-12-03 03:55:00 UTC,false,false,true
"""CI210""","""wind_speed""",1,false,0.669,2024-01-01 00:00:00 UTC,true,null,"""m/s""",25348,"""Carmel""",75.0,36.54,-121.88,66,"""CA""","""America/Los_Angeles""",45.9,3.89,2009-01-28 00:00:00 UTC,2024-12-03 01:00:00 UTC,false,false,true
"""D4637""","""wind_speed""",1,false,0.0,2024-01-01 00:00:00 UTC,true,null,"""m/s""",27490,"""DW4637 Pebble Beach""",92.0,36.56951,-121.9676,65,"""CA""","""America/Los_Angeles""",75.5,6.75,2010-04-28 00:00:00 UTC,2024-07-09 13:00:00 UTC,false,false,false
"""CI229""","""wind_speed""",1,false,0.514,2024-01-01 00:00:00 UTC,true,null,"""m/s""",31842,"""Laguna Seca""",320.0,36.57,-121.786389,66,"""CA""","""America/Los_Angeles""",305.1,3.74,2011-10-27 00:00:00 UTC,2024-12-03 01:00:00 UTC,false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""F9456""","""wind_speed""",1,false,0.0,2024-01-01 00:00:00 UTC,true,null,"""m/s""",162574,"""FW9456 Pebble Beach""",157.0,36.56717,-121.958,65,"""CA""","""America/Los_Angeles""",183.7,6.27,2021-03-22 19:05:00 UTC,2024-12-03 04:05:00 UTC,false,false,true
"""G0246""","""wind_speed""",1,false,0.0,2024-01-01 00:00:00 UTC,true,null,"""m/s""",166603,"""GW0246 Monterey""",314.0,36.57217,-121.7975,65,"""CA""","""America/Los_Angeles""",301.8,3.11,2021-07-26 22:09:00 UTC,2024-12-03 04:02:00 UTC,false,false,true
"""G2403""","""wind_speed""",1,false,0.0,2023-12-31 23:59:00 UTC,true,null,"""m/s""",178964,"""GW2403 Salinas""",1190.0,36.52083,-121.7,65,"""CA""","""America/Los_Angeles""",1210.6,9.56,2022-10-18 20:10:00 UTC,2024-12-03 04:04:00 UTC,false,false,true


In [5]:
s.QC_SUMMARY["TOTAL_OBSERVATIONS_FLAGGED"], (~s.df()["qc_passed"]).sum()


(3, 3)

In [11]:
s.json["STATION"]

[{'ID': '276',
  'STID': 'KMRY',
  'NAME': 'Monterey Regional Airport',
  'ELEVATION': '167.0',
  'LATITUDE': '36.59047',
  'LONGITUDE': '-121.84875',
  'STATUS': 'ACTIVE',
  'MNET_ID': '1',
  'STATE': 'CA',
  'TIMEZONE': 'America/Los_Angeles',
  'ELEV_DEM': '170.6',
  'DISTANCE': 0.0,
  'PERIOD_OF_RECORD': {'start': '1997-04-12T00:00:00Z',
   'end': '2024-12-03T04:00:00Z'},
  'UNITS': {'position': 'm', 'elevation': 'ft'},
  'QC_FLAGGED': False,
  'RESTRICTED': False},
 {'ID': '306',
  'STID': 'KOAR',
  'NAME': 'FORT ORD/FRITZSCHE',
  'ELEVATION': '135.0',
  'LATITUDE': '36.68',
  'LONGITUDE': '-121.77',
  'STATUS': 'ACTIVE',
  'MNET_ID': '1',
  'STATE': 'CA',
  'TIMEZONE': 'America/Los_Angeles',
  'ELEV_DEM': '144.4',
  'DISTANCE': 7.57,
  'PERIOD_OF_RECORD': {'start': '2020-11-04T19:04:00Z',
   'end': '2024-12-03T03:55:00Z'},
  'UNITS': {'position': 'm', 'elevation': 'ft'},
  'QC_FLAGGED': False,
  'RESTRICTED': False},
 {'ID': '25348',
  'STID': 'CI210',
  'NAME': 'Carmel',
  'ELEVA

In [7]:
for i in s.STATION:
    if i["STID"] == "E1554":
        assert i["OBSERVATIONS"] == {
            "wind_speed_value_1": {
                "value": 0.0,
                "date_time": "2024-01-01T00:00:00Z",
                "qc": {"status": "failed", "qc_flags": [3]},
            }
        }
        E1554 = s.df().filter(stid="E1554")
        assert E1554["date_time"][0] == datetime(2024, 1, 1, 0, 0, tzinfo=UTC)
        assert E1554["variable"][0] == "wind_speed"
        assert E1554["value"][0] == 0
        assert E1554["qc_flags"].to_list() == [[3]]
        assert not E1554["qc_passed"][0]


KeyError: 'OBSERVATIONS'